<a href="https://colab.research.google.com/github/jmical-mc/dw_matrix_road_sign/blob/master/day4_matrix3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [5]:
len(np.unique(y_train))

43

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

    Flatten(),

    Dense(num_classes, activation='softmax')
  ])

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )

  return model

In [9]:
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 12ms/step - loss: 49.3642 - accuracy: 0.7082 - val_loss: 0.2790 - val_accuracy: 0.9319
Epoch 2/5
272/272 [==============================] - 3s 10ms/step - loss: 0.2228 - accuracy: 0.9453 - val_loss: 0.1441 - val_accuracy: 0.9679
Epoch 3/5
272/272 [==============================] - 3s 10ms/step - loss: 0.1341 - accuracy: 0.9661 - val_loss: 0.0948 - val_accuracy: 0.9769
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.0901 - accuracy: 0.9772 - val_loss: 0.0622 - val_accuracy: 0.9859
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.0770 - accuracy: 0.9821 - val_loss: 0.0494 - val_accuracy: 0.9879


In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [13]:
predict(model_trained, X_test, y_test)

0.8308390022675737

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [15]:
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

train_and_predict(get_cnn_v2(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 5.4167 - accuracy: 0.3319 - val_loss: 1.0938 - val_accuracy: 0.6985
Epoch 2/5
272/272 [==============================] - 3s 10ms/step - loss: 1.0479 - accuracy: 0.6863 - val_loss: 0.3501 - val_accuracy: 0.9163
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 0.5503 - accuracy: 0.8318 - val_loss: 0.1588 - val_accuracy: 0.9595
Epoch 4/5
272/272 [==============================] - 3s 10ms/step - loss: 0.3690 - accuracy: 0.8878 - val_loss: 0.1109 - val_accuracy: 0.9786
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.2721 - accuracy: 0.9172 - val_loss: 0.0673 - val_accuracy: 0.9855


0.9138321995464853

In [16]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

train_and_predict(get_cnn_v3(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 1.6732 - accuracy: 0.6230 - val_loss: 0.2066 - val_accuracy: 0.9545
Epoch 2/5
272/272 [==============================] - 3s 13ms/step - loss: 0.3460 - accuracy: 0.9003 - val_loss: 0.0810 - val_accuracy: 0.9823
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1927 - accuracy: 0.9443 - val_loss: 0.0459 - val_accuracy: 0.9907
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1351 - accuracy: 0.9605 - val_loss: 0.0258 - val_accuracy: 0.9940
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1073 - accuracy: 0.9691 - val_loss: 0.0152 - val_accuracy: 0.9961


0.971655328798186

In [17]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

train_and_predict(get_cnn_v4(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.5444 - accuracy: 0.3212 - val_loss: 0.9136 - val_accuracy: 0.7478
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.8098 - accuracy: 0.7501 - val_loss: 0.2079 - val_accuracy: 0.9423
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3822 - accuracy: 0.8838 - val_loss: 0.0920 - val_accuracy: 0.9724
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2410 - accuracy: 0.9291 - val_loss: 0.0704 - val_accuracy: 0.9809
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1779 - accuracy: 0.9459 - val_loss: 0.0521 - val_accuracy: 0.9858


0.9517006802721089

In [18]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.9114 - accuracy: 0.2280 - val_loss: 1.3284 - val_accuracy: 0.5648
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 1.0496 - accuracy: 0.6634 - val_loss: 0.3629 - val_accuracy: 0.9054
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4427 - accuracy: 0.8606 - val_loss: 0.1212 - val_accuracy: 0.9673
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2573 - accuracy: 0.9219 - val_loss: 0.0564 - val_accuracy: 0.9826
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1979 - accuracy: 0.9410 - val_loss: 0.0428 - val_accuracy: 0.9878


0.9678004535147392

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray(X_test).reshape(-1, 32, 32, 1)

In [31]:
model = get_cnn_v5((32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train)
predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.7431 - accuracy: 0.2433 - val_loss: 0.6509 - val_accuracy: 0.8008
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.5279 - accuracy: 0.8317 - val_loss: 0.1456 - val_accuracy: 0.9533
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2367 - accuracy: 0.9264 - val_loss: 0.0547 - val_accuracy: 0.9840
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1608 - accuracy: 0.9489 - val_loss: 0.0398 - val_accuracy: 0.9900
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1226 - accuracy: 0.9620 - val_loss: 0.0240 - val_accuracy: 0.9935


0.9673469387755103